## Project – Creating an automated trading system

#### Christian Karvonen IA-15 & Wilhelm Kinos IA-15

#### Imports:

In [1]:
import datetime as dt
import numpy as np
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
from pandas_datareader.data import DataReader
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.linear_model import LinearRegression, ElasticNetCV, Ridge, BayesianRidge, LassoCV, LassoLarsCV
from sklearn.neural_network import MLPRegressor
from sklearn import preprocessing, svm, model_selection
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import confusion_matrix
init_notebook_mode(connected=True)

### All definitions/functions:

#### Rolling average definition:

In [2]:
def rolling_average(values, window):
    nan = []
    nan = np.append(nan, np.repeat(np.nan, window-1))
    weights = np.repeat(1.0, window)/window
    sma = np.convolve(values,weights,'valid')
    sma = np.append(nan, sma)
    return sma

#### MASE

In [3]:
def MASE(y_test, forecast_prediction):
    y_test, forecast_prediction = np.array(y_test), np.array(forecast_prediction)
    n = len(y_test)
    d = np.abs(np.diff(y_test, axis=0)).sum()/(n-1) 
    errors = np.abs(y_test.flat - forecast_prediction)
    return errors.mean()/d

#### SMAPE

In [4]:
def SMAPE(y_test, forecast_prediction):
    y_test, forecast_prediction = np.array(y_test), np.array(forecast_prediction)
    return 1/len(y_test) * np.sum(np.abs(forecast_prediction - y_test.flat) / (np.abs(y_test) + np.abs(forecast_prediction))/2)

#### Defining timeframe of stocks:

In [5]:
end = dt.date.today()
endday = end
endday = endday.strftime('%Y-%m-%d')
print([endday])
start = end - dt.timedelta(days=5*365)

['2018-11-05']


#### Use of DataReader to get MU stock data from IEX https://iextrading.com/developer/

In [7]:
df = DataReader('MU', 'iex', start, end)
df.tail(10)

5y


,open,high,low,close,volume
date,,,,,
2018-10-22,40.56,40.70,38.640,39.76,34441221
2018-10-23,38.15,39.11,37.410,38.68,46364174
2018-10-24,38.00,38.30,35.400,35.43,53096450
2018-10-25,35.59,37.16,35.060,36.78,39625699
2018-10-26,35.28,35.91,34.300,35.40,41257357
2018-10-29,36.14,36.20,33.820,34.66,34176423
2018-10-30,34.64,36.05,34.495,36.01,35319497
2018-10-31,36.58,38.04,35.800,37.72,44055370
2018-11-01,37.98,40.20,37.520,40.12,43981410


### The 5 features:

#### Momentum: 

In [8]:
df['momentum'] = df.close - df.close.shift(4)
df[['momentum']].head(10)

,momentum
date,
2013-11-06,NaN
2013-11-07,NaN
2013-11-08,NaN
2013-11-11,NaN
2013-11-12,1.130
2013-11-13,1.201
2013-11-14,1.085
2013-11-15,1.191
2013-11-18,0.660


#### A/O Oscillator (accumulation/distribution oscillator):

In [9]:
df['A/O'] = (df.high - df.close.shift(1))/(df.high - df.low)
df[['A/O']].head(10)

,A/O
date,
2013-11-06,NaN
2013-11-07,1.139130
2013-11-08,0.786275
2013-11-11,0.460526
2013-11-12,0.727273
2013-11-13,0.631579
2013-11-14,0.933333
2013-11-15,1.049180
2013-11-18,0.530380


#### MA5:

In [10]:
df['MA5'] = rolling_average(df.close, 5)
df[['MA5']].head(10)

,MA5
date,
2013-11-06,NaN
2013-11-07,NaN
2013-11-08,NaN
2013-11-11,NaN
2013-11-12,18.0448
2013-11-13,18.3288
2013-11-14,18.6230
2013-11-15,18.8942
2013-11-18,19.0982


#### BIAS6:

In [11]:
df['MA6'] = rolling_average(df.close, 6)
df['BIAS6'] = ((df.close-df.MA6)/(df.MA6))*100
df[['BIAS6']].head(10)

,BIAS6
date,
2013-11-06,NaN
2013-11-07,NaN
2013-11-08,NaN
2013-11-11,NaN
2013-11-12,NaN
2013-11-13,4.009382
2013-11-14,3.885089
2013-11-15,3.721930
2013-11-18,1.739612


#### ROC (rate of change)

In [12]:
df['ROC'] = (df.close - df.close.shift(3)) / df.close.shift(3) * 100
df[['ROC']].head(10)

,ROC
date,
2013-11-06,NaN
2013-11-07,NaN
2013-11-08,NaN
2013-11-11,4.400000
2013-11-12,5.141374
2013-11-13,4.501519
2013-11-14,5.035577
2013-11-15,4.460548
2013-11-18,1.955603


In [13]:
df.head(5)

,open,high,low,close,volume,momentum,A/O,MA5,MA6,BIAS6,ROC
date,,,,,,,,,,,
2013-11-06,17.77,17.820,17.39,17.500,21799539,NaN,NaN,NaN,NaN,NaN,NaN
2013-11-07,17.83,18.155,17.58,17.719,54196790,NaN,1.139130,NaN,NaN,NaN,NaN
2013-11-08,17.70,18.120,17.61,18.105,33201332,NaN,0.786275,NaN,NaN,NaN,NaN
2013-11-11,18.04,18.280,17.90,18.270,36990366,NaN,0.460526,NaN,NaN,NaN,4.400000
2013-11-12,18.17,18.670,18.12,18.630,32232323,1.13,0.727273,18.0448,NaN,NaN,5.141374


### Moving window

In [14]:
def get_moving_window(data, window):
    for i in range(1, window + 1):
        df["Moving_Window_"+str(i)] = data.shift(i) 

In [15]:
#Label
label_int = int(1)
df['label'] = df['close'].shift(-label_int)

In [16]:
#df = df.drop(['open','high','low','volume','momentum','A/O','MA5','MA6','BIAS6','ROC'],1)

In [17]:
get_moving_window(df.close, 15)
df.head(10)

,open,high,low,close,volume,momentum,A/O,MA5,MA6,BIAS6,...,Moving_Window_6,Moving_Window_7,Moving_Window_8,Moving_Window_9,Moving_Window_10,Moving_Window_11,Moving_Window_12,Moving_Window_13,Moving_Window_14,Moving_Window_15
date,,,,,,,,,,,,,,,,,,,,,
2013-11-06,17.77,17.820,17.39,17.500,21799539,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-11-07,17.83,18.155,17.58,17.719,54196790,NaN,1.139130,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-11-08,17.70,18.120,17.61,18.105,33201332,NaN,0.786275,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-11-11,18.04,18.280,17.90,18.270,36990366,NaN,0.460526,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-11-12,18.17,18.670,18.12,18.630,32232323,1.130,0.727273,18.0448,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-11-13,18.47,18.990,18.42,18.920,38821218,1.201,0.631579,18.3288,18.190667,4.009382,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-11-14,18.91,19.200,18.90,19.190,31417954,1.085,0.933333,18.6230,18.472333,3.885089,...,17.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-11-15,19.38,19.830,19.22,19.461,51592829,1.191,1.049180,18.8942,18.762667,3.721930,...,17.719,17.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-11-18,19.58,19.880,19.09,19.290,47549892,0.660,0.530380,19.0982,18.960167,1.739612,...,18.105,17.719,17.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Dropna

In [18]:
df.dropna(inplace=True)
df.head()

,open,high,low,close,volume,momentum,A/O,MA5,MA6,BIAS6,...,Moving_Window_6,Moving_Window_7,Moving_Window_8,Moving_Window_9,Moving_Window_10,Moving_Window_11,Moving_Window_12,Moving_Window_13,Moving_Window_14,Moving_Window_15
date,,,,,,,,,,,,,,,,,,,,,
2013-11-27,20.41,21.19,20.29,21.170,40572662,1.180,0.977778,20.3040,20.053333,5.568484,...,19.06,19.29,19.461,19.190,18.920,18.630,18.270,18.105,17.719,17.500
2013-11-29,21.30,21.31,20.90,21.100,20081775,0.910,0.341463,20.5260,20.436667,3.245800,...,18.80,19.06,19.290,19.461,19.190,18.920,18.630,18.270,18.105,17.719
2013-12-02,21.10,21.64,20.96,21.270,34102762,1.410,0.794118,20.7420,20.650000,3.002421,...,19.99,18.80,19.060,19.290,19.461,19.190,18.920,18.630,18.270,18.105
2013-12-03,21.44,21.90,21.33,21.830,43553080,1.520,1.105263,21.1360,20.923333,4.333280,...,20.19,19.99,18.800,19.060,19.290,19.461,19.190,18.920,18.630,18.270
2013-12-04,21.79,21.93,21.41,21.759,34609036,0.589,0.192308,21.4258,21.239833,2.444307,...,19.86,20.19,19.990,18.800,19.060,19.290,19.461,19.190,18.920,18.630


Linear Regression

In [19]:
def get_y_columns(data):
    feature_col = 12
    return data.columns[feature_col:].values 
    
print(get_y_columns(df))
print(np.array(df[get_y_columns(df)]))

['Moving_Window_1' 'Moving_Window_2' 'Moving_Window_3' 'Moving_Window_4'
 'Moving_Window_5' 'Moving_Window_6' 'Moving_Window_7' 'Moving_Window_8'
 'Moving_Window_9' 'Moving_Window_10' 'Moving_Window_11'
 'Moving_Window_12' 'Moving_Window_13' 'Moving_Window_14'
 'Moving_Window_15']
[[20.31  19.86  20.19  ... 18.105 17.719 17.5  ]
 [21.17  20.31  19.86  ... 18.27  18.105 17.719]
 [21.1   21.17  20.31  ... 18.63  18.27  18.105]
 ...
 [34.66  35.4   36.78  ... 41.97  41.61  42.27 ]
 [36.01  34.66  35.4   ... 42.47  41.97  41.61 ]
 [37.72  36.01  34.66  ... 42.34  42.47  41.97 ]]


In [20]:
x = np.array(df.drop(['label'], 1))
y = np.array(df['label'])
print(x)
print(y)
y = y.reshape(-1, 1)

scaler = preprocessing.StandardScaler().fit(y)

x = scaler.transform(x)
y = scaler.transform(y)

tscv = TimeSeriesSplit(n_splits=5)

for train_index, test_index in tscv.split(x):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]

[[20.41  21.19  20.29  ... 18.105 17.719 17.5  ]
 [21.3   21.31  20.9   ... 18.27  18.105 17.719]
 [21.1   21.64  20.96  ... 18.63  18.27  18.105]
 ...
 [34.64  36.05  34.495 ... 41.97  41.61  42.27 ]
 [36.58  38.04  35.8   ... 42.47  41.97  41.61 ]
 [37.98  40.2   37.52  ... 42.34  42.47  41.97 ]]
[21.1  21.27 21.83 ... 37.72 40.12 40.32]


In [21]:
regr = LinearRegression()
regr.fit(x_train, y_train.reshape(len(y_train)))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [25]:
regr.score(x_test, y_test)
forecast = regr.predict(x_test)
#inverse_transform the result for future plotting
forecast_list = forecast.tolist()
new_forecast = scaler.inverse_transform(forecast_list)

In [23]:
MASE(y_test, forecast)

1.0218069996354244

In [24]:
SMAPE(y_test, forecast)

2.9569135784540905

In [27]:
dates = df.index.values
print([endday])
datespred = np.append(dates, [endday])
print(datespred)
lenofytest = len(y_test)
print(lenofytest)

['2018-11-05']
['2013-11-27' '2013-11-29' '2013-12-02' ... '2018-10-31' '2018-11-01'
 '2018-11-05']
207


In [31]:
df_cm = pd.DataFrame(index=datespred[-lenofytest:])
#actual
df_cm['actual'] = df['close'][-lenofytest:]
#label
df_cm['label'] = df['label'][-lenofytest:]
#pred
df_cm['pred'] = new_forecast
#Creating the buy and the sell decisions for label
df_cm['labelDecision'] = np.where(df_cm['label'] > df_cm['actual'], "Buy", "Sell")
#Creating the buy and the sell decisions for pred
df_cm['predDecision'] = np.where(df_cm['pred'] > df_cm['actual'], "Buy", "Sell")
#Determining the hits between labelDecision and predDecisions
df_cm['hitDecision'] = np.where(df_cm['labelDecision'] == df_cm['predDecision'], 1.0, 0.0)

#Finding confusion matrix values TN, FP, FN and TP
df_cm['TN'] = np.where((df_cm['labelDecision'] == "Sell") & (df_cm['predDecision'] == "Sell"), 1.0, 0.0)
df_cm['FP'] = np.where((df_cm['labelDecision'] == "Sell") & (df_cm['predDecision'] == "Buy"), 1.0, 0.0)
df_cm['FN'] = np.where((df_cm['labelDecision'] == "Buy") & (df_cm['predDecision'] == "Sell"), 1.0, 0.0)
df_cm['TP'] = np.where((df_cm['labelDecision'] == "Buy") & (df_cm['predDecision'] == "Buy"), 1.0, 0.0)
df_cm.head(10)

,actual,label,pred,labelDecision,predDecision,hitDecision,TN,FP,FN,TP
2018-01-10,43.31,42.82,42.560898,Sell,Sell,1.0,1.0,0.0,0.0,0.0
2018-01-11,42.82,42.81,43.519285,Sell,Buy,0.0,0.0,1.0,0.0,0.0
2018-01-12,42.81,42.92,42.946135,Buy,Buy,1.0,0.0,0.0,0.0,1.0
2018-01-16,42.92,44.26,43.359918,Buy,Buy,1.0,0.0,0.0,0.0,1.0
2018-01-17,44.26,43.99,42.921978,Sell,Sell,1.0,1.0,0.0,0.0,0.0
2018-01-18,43.99,42.75,44.661416,Sell,Buy,0.0,0.0,1.0,0.0,0.0
2018-01-19,42.75,42.88,43.362473,Buy,Buy,1.0,0.0,0.0,0.0,1.0
2018-01-22,42.88,43.95,42.688210,Buy,Sell,0.0,0.0,0.0,1.0,0.0
2018-01-23,43.95,43.08,42.437603,Sell,Sell,1.0,1.0,0.0,0.0,0.0
2018-01-24,43.08,43.01,44.464503,Sell,Buy,0.0,0.0,1.0,0.0,0.0


In [35]:
#printing out the confusion matrix values 
print(len(df_cm.loc[df_cm.TN == 1.0]))
print(len(df_cm.loc[df_cm.FP == 1.0]))
print(len(df_cm.loc[df_cm.FN == 1.0]))
print(len(df_cm.loc[df_cm.TP == 1.0]))


#checking if the values are correct with pd.crosstab
df_confusion = pd.crosstab(df_cm.labelDecision, df_cm.predDecision)
df_confusion

48
54
44
61


predDecision,Buy,Sell
labelDecision,,
Buy,61,44
Sell,54,48


In [32]:
#hitRatio 1
lenOfHits = len(df_cm.loc[df_cm.hitDecision == 1.0])
lenOfMisses = len(df_cm.loc[df_cm.hitDecision == 0.0])
lenOfDecisions = len(df_cm.hitDecision)

print(lenOfHits)
print(lenOfMisses)
print(lenOfDecisions)

hitRatio = ((lenOfHits) / (lenOfDecisions)) * 100
print(hitRatio)

109
98
207
52.65700483091788
